# Предварительная настройка среды разработки

## Google Collab

### Подключение сервисов Google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Setup Complete")

### Установка необходимых библиотек

## Локальная среда

### Проверка доступности GPU

Чтобы получить GPU, нажмите _Runtime_ -> _Change runtime type_, затем измените _Hardware accelerator_ с _None_ на _GPU_.

Мы можем проверить, что нам назначен GPU, и просмотреть его характеристики:

In [1]:
!nvidia-smi

Wed Jan 11 15:49:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.92.01    Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8     6W /  74W |      9MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Установка необходимых библиотек

In [12]:
!conda install -y -c conda-forge transformers
!conda install -y -c conda-forge datasets
!conda install -y -c conda-forge librosa
!conda install -c conda-forge ffmpeg
!conda install -c conda-forge huggingface_hub

!pip install --no-input evaluate
!pip install --no-input jiwer
!pip install --no-input gradio

print("Library installation complete!")

Solving environment: done

# All requested packages already installed.

Library installation complete!


### Загрузка необходимых модулей

In [22]:
from huggingface_hub import notebook_login
from datasets import load_dataset, DatasetDict

print("Required libraries/modules initialization complete!")

Required libraries/modules initialization complete!


### Настройка окружения

In [15]:
# Аутентификация ноутбука в HuggingFace HUB
notebook_login()

print("Environment setup completed!")

Environment setup completed!


# Загрузка набора данных

Используя 🤗 Datasets, загрузка и подготовка данных чрезвычайно проста. 

Мы можем загрузить и подготовить сплиты Common Voice всего одной строкой кода. 

Во-первых, убедитесь, что вы приняли условия использования на Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). После принятия условий вы получите полный доступ к набору данных и сможете загружать данные локально.

Мы объединим `train` и `validation` части набора данных, чтобы получить как можно больше данных для обучения. Мы будем использовать
данных `test` в качестве нашего тестового набора:

In [33]:
# Создадим словарь для хранения датасета
common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_2_0", "ru", split="train+validation", cache_dir="DATA/CACHE", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_2_0", "ru", split="test", cache_dir="DATA/CACHE", use_auth_token=True)

print(common_voice)

Found cached dataset common_voice_2_0 (/home/artyom/Whisper_Train/DATA/CACHE/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079)
Found cached dataset common_voice_2_0 (/home/artyom/Whisper_Train/DATA/CACHE/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3898
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1561
    })
})


Большинство наборов данных ASR предоставляют только входные образцы аудио ('audio') и соответствующий транскрибированный текст ('sentence').  Common Voice содержит дополнительную метаданные, такие как `accent` и `locale`, которые мы можем игнорировать для ASR.

Оставив блокнот максимально общим, мы рассматриваем только входной звук и транскрибированный текст для тонкой настройки, отбрасывая дополнительную информацию метаданных:

In [34]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3898
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1561
    })
})


# Подготовка экстрактора признаков, токенизатора и данных

# Полезные ссылки

- [Fine-Tune Whisper For Multilingual ASR with Transformers (Статья)](https://huggingface.co/blog/fine-tune-whisper)